# Optimal Predictor Machine used with Mushroom.csv

Load required libraries and functions

In [1]:
library('data.table')
library('extraDistr')
library('foreach')
library('png')
source('tplotfunctions.R')
source('guessmetadata.R')
source('buildagent.R')
source('infer.R')
source('decide.R')
source('mutualinfo.R')
source('rF.R')
source('plotFsamples1D.R')
library(reticulate)
options(repr.plot.width=10*sqrt(2), repr.plot.height=10)

Warning message in match.arg(palette, names(.schemes), several.ok = FALSE):
"strings not representable in native encoding will be translated to UTF-8"


## Changing the csv file to fit with optimal machine
Here som changes are done to make sure that the optimal machine will run with the dataset. This includes limmiting the amount of data, as well as limmiting variates.
The code bellow includes the most variates possible.
Further cuts will be made later.


In [2]:
# Set the file path to your CSV file
file_path <- "../Datasett/mushrooms.csv"

# Set the full path for the output CSV file
output_file_path <- "../Datasett/modified_data.csv"

columns_to_keep <- c("class","capShape","capSurface","bruises","gillAttachment","gillSpacing","gillSize","stalkShape","stalkSurfaceAboveRing","stalkSurfaceBelowRing","veilColor","ringNumber","ringType")
#gillColor` has 12, `capColor` 10, `stalkColorBelowRing` 9, `odor` 9, `stalkColorAboveRing` 8, `sporePrintColor` 9, population, habitat
# Subset the data to keep only the selected columns

data <- read.csv(file_path, header = TRUE, sep = ",", na.strings = "?")
# Show the number of unique values for each attribute
unique_counts <- sapply(data, function(x) length(unique(x)))
print(unique_counts)


# Load the CSV file with comma as the separator and the first row as headers
data <- data[, columns_to_keep]

# Save the modified data as a CSV file with the specified full path
write.csv(data, file = output_file_path, row.names = FALSE)

# Show the number of unique values for each attribute
unique_counts <- sapply(data, function(x) length(unique(x)))
print(unique_counts)

                class              capShape            capSurface 
                    2                     6                     4 
             capColor               bruises                  odor 
                   10                     2                     9 
       gillAttachment           gillSpacing              gillSize 
                    2                     2                     2 
            gillColor            stalkShape             stalkRoot 
                   12                     2                     5 
stalkSurfaceAboveRing stalkSurfaceBelowRing   stalkColorAboveRing 
                    4                     4                     9 
  stalkColorBelowRing              veilType             veilColor 
                    9                     1                     4 
           ringNumber              ringType       sporePrintColor 
                    3                     5                     9 
           population               habitat 
                 

Based in the info bellow we can se the number of possible values for each variate: `gillColor` has 12, `capColor` 10, `stalkColorBelowRing` 9, `odor` 9, `stalkColorAboveRing` 9, `sporePrintColor` 9, `habitat` 7, `population` 6, `capShape` 6

The `buildagent()` method can't run without limmiting the training set.
Here we are building an temorary agent for calcualting predictors relevance.

In [3]:
guessmetadata(data = "../Datasett/modified_data.csv",
              file = "../Datasett/preliminary.csv")

opmall <- buildagent(metadata = "../Datasett/preliminary.csv",
                     data = fread("../Datasett/modified_data.csv"))

For checking correlation between to variates

----

## Calculating relevance of predictors

The agent can calculate the mutual information (measured in shannons) between any two sets of variates `A` and `B` of our choice, with the function `mutualinfo(probs, A, B)`. The joint probabilities for `(A,B)` must first be calculated with `infer()`.

For instance, what is the mutual information between `capColor` and `dodor`? The mutual information in this case can be anywhere between 0 Sh and 1 Sh 

In this example we will compare the fulle sett but only removing one variate at a time.
When the score in sh is lower, it will mean that the variate left out has informasjon which when combined with the other variates has a significant impact in the prediction of the `class`.


# The code bellow checks the relevance with combination of two varaites.

In [ ]:
navn <- colnames(data)
navn <- navn[2:15]
print(length(navn))

i <- 1
j <- 2

result_list <- list()

while (i != 13) {
  while (j != 14) {

    probs <- infer(agent = opmall, predictand = c("class", navn[i], navn[j]))
    result <- mutualinfo(probs = probs, A = "class", B = c(navn[i], navn[j]))

    result_list[[paste0(navn[i], " & ", navn[j])]] <- result
    #print(paste0(signif(result, 4), ' Sh'))

    j <- j + 1
  }
  j <- i + 2
  i <- i + 1
}
sorted_result_list <- result_list[order(-unlist(result_list))]
sorted_result_list


# Test of each individual variate to check their relevance to `class`

In [ ]:
## list of all variates
variates <- names(dimnames(opmall$counts))

## list of all variates except 'income'
predictors <- variates[variates != 'class']

## prepare vector to contain the mutual information
relevances <- numeric(length(predictors))
names(relevances) <- predictors

## calculate, for each variate,
## the joint probability distribution 'probs'
## and then mutual information 'relevance' (in shannons)
## between 'income' and that variate
for(var in predictors){
    probs <- infer(agent=opmall, predictand=c('class',var))
    relevances[var] <- mutualinfo(probs=probs, A='class', B=var)
}

## output the mutual informations in decreasing order
sort(relevances, decreasing=TRUE)

# Checking the relevance for all varaiates except one.

In [ ]:
navn <- colnames(data)
navn <- navn[2:15]
print(length(navn))
i <- 1
result_list <- list()

while (i <= 14) {
  print(navn[i])
  print(i)
  # Create a modified version of navn without the current_variate
  navn_without_current <- navn[-i]
  probs <- infer(agent = opmall, predictand = c("class", navn_without_current))
  result <- mutualinfo(probs = probs, A = "class", B = navn_without_current)

  result_list[[paste0("Uten variate; ", navn[i])]] <- result
  i <- i + 1
}
sorted_result_list <- result_list[order(-unlist(result_list))]
sorted_result_list

# Loadign inn the train_set.csv

In [4]:
# Set the file path to your CSV file
file_path <- "../Datasett/train_set.csv"

# Load the CSV file with comma as the separator and the first row as headers
data <- read.csv(file_path, header = TRUE, sep = ",", na.strings = "?")

Only keeping:   "class","capShape","capSurface","bruises","gillAttachment", "gillSpacing", "gillSize","stalkShape","stalkSurfaceAboveRing","stalkSurfaceBelowRing","veilColor","ringNumber","ringType"


In [5]:
# Specify the columns to keep
columns_to_keep <- c("class","capShape","capSurface","bruises","gillAttachment", "gillSpacing", "gillSize","stalkShape","stalkSurfaceAboveRing","stalkSurfaceBelowRing","veilColor","ringNumber","ringType")
# Subset the data to keep only the selected columns
data <- data[, columns_to_keep]
output_file_path <- "../Datasett/train_modified.csv"
write.csv(data, file = output_file_path, row.names = FALSE)

Checking to see the amount of unique rows in the dataset.

In [6]:
# Assuming your data frame is named "data"
unique_rows <- !duplicated(data)

# Print the number of unique rows
print(paste0("Number of unique rows: ", nrow(data[unique_rows,])))


[1] "Number of unique rows: 180"


Creating matadatafile for the datasett. This tells us how many variates there are, as well as how many possible values each variate can have

In [7]:
guessmetadata(data = '../Datasett/train_modified.csv',
              file = "../Datasett/preliminary.csv")

## Building the agent

is done with the function `buildagent()`, which takes as input the metadata and training data, and output an object of class "agent".

In this exploration we build two agents:
- `opm10`: trained with 10 datapoints
- `opmall`: trained with all datapoints

In [8]:
#opm10 <- buildagent(metadata='meta_income_data_example.csv',
#                    data=fread('train-income_data_example.csv', header=TRUE)[1:10])
#
#opmall <- buildagent(metadata='meta_income_data_example.csv',
#                     data='train-income_data_example.csv')
opm10 <- buildagent(metadata = "../Datasett/preliminary.csv",
                     data = fread('../Datasett/train_modified.csv')[1:10])

opmall <- buildagent(metadata = "../Datasett/preliminary.csv",
                     data = fread('../Datasett/train_modified.csv'))

Checking for how many total variates times the amount of data.????

In [9]:
# Set the file path to your CSV file
file_path <- "../Datasett/test_set.csv"

# Load the CSV file with comma as the separator and the first row as headers
test <- read.csv(file_path, header = TRUE, sep = ",", na.strings = "?")

# Specify the columns to keep
columns_to_keep <- c("class","capShape","capSurface","bruises","gillAttachment", "gillSpacing", "gillSize","stalkShape","stalkSurfaceAboveRing","stalkSurfaceBelowRing","veilColor","ringNumber","ringType")

# Subset the data to keep only the selected columns
test <- test[, columns_to_keep]

In [10]:
# Specify the columns to keep
columns_to_keep <- c("class","capShape","capSurface","bruises","gillAttachment", "gillSpacing", "gillSize","stalkShape","stalkSurfaceAboveRing","stalkSurfaceBelowRing","veilColor","ringNumber","ringType")

# Subset the data to keep only the selected columns
data <- data[, columns_to_keep]
output_file_path <- "../Datasett/train_modified.csv"
write.csv(data, file = output_file_path, row.names = FALSE)

guessmetadata(data = '../Datasett/train_modified.csv',
              file = "../Datasett/preliminary.csv")

opm10 <- buildagent(metadata = "../Datasett/preliminary.csv",
                     data = fread('../Datasett/train_modified.csv')[1:10])

opmall <- buildagent(metadata = "../Datasett/preliminary.csv",
                     data = fread('../Datasett/train_modified.csv'))

In [13]:
mushroom_utilities <- matrix(c(1,0, 0.7, 0.8), nrow=2, byrow=TRUE, dimnames=list(action=c('Eat','Not-eat'), class=c('e', 'p')))

print(mushroom_utilities) 

         class
action      e   p
  Eat     1.0 0.0
  Not-eat 0.7 0.8


In [14]:
antall <- nrow(test)
correct_list <- list()
result_list <- list()
confusionmatrix <- mushroom_utilities * 0L

i <- 1
while (i <= antall) {
    result <- infer(
        agent = opmall, 
        predictand = "class",
        predictor = list(capShape = test[i,2], capSurface= test[i,3],bruises=test[i,4], gillAttachment = test[i,5], gillSpacing = test[i,6], gillSize = test[i,7], stalkShape = test[i,8], stalkSurfaceAboveRing = test[i, 9], stalkSurfaceBelowRing = test[i,10], veilColor = test[i, 11], ringNumber = test[i, 12], ringType = test[i,13])
    )

    decision <- decide(utils = mushroom_utilities, probs = result)$optimal
    if(decision == "e") {
        decision = "Eat"
    }
    else {
        decision = "Not-eat"
    }
    trueclass <- test[i, "class"]

    confusionmatrix[decision, trueclass] <- confusionmatrix[decision, trueclass] + 1L
    
    i <- i + 1  # Increment the loop counter
}

print(confusionmatrix)


         class
action      e   p
  Eat     825   0
  Not-eat  18 782


In [ ]:
mushroom_utilities <- matrix(c(1,0, 0.2,0.2), nrow=2, byrow=TRUE, dimnames=list(action=c('Eat','Not-eat'), class=c('e', 'p')))

print(mushroom_utilities) 

In [ ]:
antall <- nrow(test)
correct_list <- list()
result_list <- list()
confusionmatrix <- mushroom_utilities * 0L

i <- 1
while (i <= antall) {
    result <- infer(
        agent = opmall, 
        predictand = "class",
        predictor = list(capShape = test[i,2], capSurface= test[i,3],bruises=test[i,4], gillAttachment = test[i,5], gillSpacing = test[i,6], gillSize = test[i,7], stalkShape = test[i,8], stalkSurfaceAboveRing = test[i, 9], stalkSurfaceBelowRing = test[i,10], veilColor = test[i, 11], ringNumber = test[i, 12], ringType = test[i,13])
    )

    decision <- decide(utils = mushroom_utilities, probs = result)$optimal
    if(decision == "e") {
        decision = "Eat"
    }
    else {
        decision = "Not-eat"
    }
    trueclass <- test[i, "class"]

    confusionmatrix[decision, trueclass] <- confusionmatrix[decision, trueclass] + 1L
    
    i <- i + 1  # Increment the loop counter
}

print(confusionmatrix)


## Secretary problem with mushroom.If the hights is not found the last is force-fed.

In [ ]:
result_secretary <- list()
truth_secretary <- list()
x <- 1

while(x <= 1000){
test <- read.csv("../Datasett/test_set.csv", header = TRUE, sep = ",", na.strings = "?")
columns_to_keep <- c("class","capShape","capSurface","bruises","gillAttachment", "gillSpacing", "gillSize","stalkShape","stalkSurfaceAboveRing","stalkSurfaceBelowRing","veilColor","ringNumber","ringType")

# Subset the data to keep only the selected columns
test <- test[, columns_to_keep]
# Shuffle the rows in the test set
test <- test[sample(nrow(test)), ]

test <- test[1:100, ]
antall <- nrow(test)
correct_list <- list()
result_list <- list()

highest <- 0
i <- 1
while (i <= antall*0.37) {
    result <- infer(
        agent = opmall, 
        predictand = "class",
        predictor = list(capShape = test[i,2], capSurface= test[i,3],bruises=test[i,4], gillAttachment = test[i,5], gillSpacing = test[i,6], gillSize = test[i,7], stalkShape = test[i,8], stalkSurfaceAboveRing = test[i, 9], stalkSurfaceBelowRing = test[i,10], veilColor = test[i, 11], ringNumber = test[i, 12], ringType = test[i,13])
    )
    decision <- decide(utils = mushroom_utilities, probs = result)$EUs

    #cat("Decision: ", decision[1], "\n")
   

    if (decision[1] > highest) {
      
      highest <- decision[1]
      #cat("Highest: ", highest, "\n")
    }
    i <- i + 1  # Increment the loop counter
}

while (i <= antall) {
      result <- infer(
        agent = opmall,
        predictand = "class",
        predictor = list(capShape = test[i,2], capSurface= test[i,3],bruises=test[i,4], gillAttachment = test[i,5], gillSpacing = test[i,6], gillSize = test[i,7], stalkShape = test[i,8], stalkSurfaceAboveRing = test[i, 9], stalkSurfaceBelowRing = test[i,10], veilColor = test[i, 11], ringNumber = test[i, 12], ringType = test[i,13])
      )
  decision <- decide(utils = mushroom_utilities, probs = result)$EUs
    if (decision[1] >= highest) {
      #print("Choosing to eat this: ")
      #print(decision)

      #print("this is the number ")
      #print(i)
      result_secretary[[x]] <- i
      truth_secretary[[x]] <- test[i, "class"]
      #print(" amongst the 100 mushroom")
      break
    }
    if (i == 100) {
      result_secretary[[x]] <- i
      truth_secretary[[x]] <- test[i, "class"]
    }
    i <- i + 1  # Increment the loop counter
}
x <- x + 1
}

double_list = list(result_secretary, truth_secretary)
output_file_path <- "../Datasett/secretary.csv"
write.csv(double_list, file = output_file_path)

#result_secretary


In [ ]:
# Install and load the ggplot2 package if you haven't already
# install.packages("ggplot2")
library(ggplot2)

# Assuming you have a list named result_secretary with 1000 elements
set.seed(123)  # Setting seed for reproducibility
#result_secretary <- sample(0:100, 1000, replace = TRUE)

# Create a data frame
data <- data.frame(result_secretary)

# Create a scatter plot using ggplot2
ggplot(data, aes(x = seq_along(result_secretary), y = result_secretary, color = factor(result_secretary == 100))) +
  geom_point() +
  scale_color_manual(values = c("green", "red"), guide = FALSE) +
  ggtitle("Scatter Plot of Result Secretary") +
  xlab("Index") +
  ylab("Result Secretary")

# To print the graph, you can run the code in the R console or R script.
# If you're using RStudio, the graph will be displayed in the Plots pane.

# This example uses color = factor(result_secretary == 100) to create a factor variable for coloring.
# The scale_color_manual function is used to specify the colors for each factor level.
# In this case, "green" for FALSE (not equal to 100) and "red" for TRUE (equal to 100).
# The guide = FALSE argument removes the color legend.


In [ ]:
i <- 1
for (value in truth_secretary) {
  if (value == "p") {
    print("POISON")
  }
  i <- i + 1
}

In [ ]:
antall <- nrow(test)
correct_list <- list()
result_list <- list()
confusionmatrix <- mushroom_utilities * 0L
i <- 1
while (i <= antall) {
    result <- infer(
    agent = opmall, predictand = "class",
    predictor = list( gillAttachment = test[i, 7], stalkShape = test[i, 11], stalkSurfaceAboveRing = test[i, 13], stalkSurfaceBelowRing = test[i, 14], veilColor = test[i, 18], ringNumber = test[i, 19], ringType = test[i, 20]) )

    decision <- decide(utils=mushroom_utilities, probs=result)
    trueclass <- test[i]$class

    confusionmatrix[decision, trueclass] <- confusionmatrix[decision, trueclass] + 1L
}
print(confusionmatrix)


In [ ]:
optimalad <- decide(utils=mushroom_utilities, probs= result_list[1])

print(optimalad)

In [ ]:
length(opmall$counts)

You can check the internal structure of the "agent" object with `str()`:

In [ ]:
str(opmall)

The "agent" object internally contains a parameter `alpha` expressing its guess of how many training data (just as a order of magnitude in base-2) would be needed to make reliable inferences about the full population.

Here are several guesses with their probabilities; you see that most probable is around 2000 training datapoints:

In [ ]:
tplot(x=opmall$alphas, y=opmall$palphas, type='b',
      xlim=c(0, 1000), ylim=c(0, NA),
      xlab='amount of needed data for overcoming prior beliefs', ylab='probability')

## Only predictands, no predictors ("unsupervised-learning mode")

Let the little-trained agent forecast `income` for next unit

In [ ]:
print(infer(agent=opm10, predictand='Na'))

The agent is internally checking *all possible* population-frequency distributions. It can therefore forecast how its probabilities could change *if more training data were provided*:

In [ ]:
plotFsamples1D(agent=opm10, n=200, predictand='class',
               ylim=c(0,1), main='opm10') # last options are for plotting

Now let's check the inference drawn by the fully-trained agent

In [ ]:
print(infer(agent=opmall, predictand='class'))

Also this agent can tell us how much would new training data change its inference:

In [ ]:
plotFsamples1D(agent=opmall, n=200, predictand='class',
               ylim=c(0,1), main='opmall')

We can draw inferences in the same way for any other variate, e.g. `capColor`.

This is the inference by the `opm10` agent:

In [ ]:
plotFsamples1D(agent=opm10, n=200, predictand='capColor',
               ylim=c(0,1), main='opm10')  # last options are for plotting

...and by the `opmall` agent:

In [ ]:
plotFsamples1D(agent=opmall, n=200, predictand='capColor',
               ylim=c(0,1), main='opmall')  # last options are for plotting

The agent can also draw inferences about several joint variates.

For instance `class` and `capColor` (let's show percentages and round to one decimal):

In [ ]:
result <- infer(agent=opmall, predictand=c('class', 'capColor'))

print(round(result*100, 1))


## Using predictors ("supervised-learning mode")

Suppose we know that `capSurface='y'` and `capColor='f'` for the new unit. What can the `class` be?

In [ ]:
#class,capShape,capSurface,capColor,bruises,odor,gillAttachment,gillSpacing,gillSize,gillColor,stalkShape,stalkRoot,stalkSurfaceAboveRing,stalkSurfaceBelowRing,stalkColorAboveRing,stalkColorBelowRing,veilType,veilColor,ringNumber,ringType,sporePrintColor,population,habitat
#p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u

In [ ]:

result <- infer(
    agent = opmall, predictand = "class",
    predictor = list(capShape = "x", capSurface = "s", bruises = "t", gillAttachment = "f", gillSize = "n", stalkShape = "e", stalkRoot = "e", stalkSurfaceAboveRing = "s", stalkSurfaceBelowRing = "s", veilType = "p", veilColor = "w", ringNumber = "o", ringType = "p")
)
#("class","capShape","capSurface","bruises","gillAttachment","gillSize","stalkShape","stalkRoot","stalkSurfaceAboveRing","stalkSurfaceBelowRing","veilType","veilColor","ringNumber","ringType")

print(round(result * 100, 1))


Also in this case we can ask how much this inference could be changed by new training data.

Note how the agent warns us that, with more training data, it could happen that the percentages would be reversed – observe the gray lines:

In [ ]:
plotFsamples1D(agent=opmall, n=600,
               predictand='class',
               predictor=list(capSurface = "y", capColor = "p"),
               ylim=c(0,1), main='opmall')

Another example: what's the probability for the possible `class`, if we know the present mushroom has  `capSurface = Black` and `capColor = p`?

In [ ]:
result <- infer(agent=opmall,
                predictand='class',
                predictor=list(capSurface = "y", capColor = "p"))

print(round(sort(result, decreasing=TRUE) * 100, 1))

plotFsamples1D(agent=opmall, n=500,
               predictand='class',
               predictor=list(capSurface = "y", capColor = "p"),
              ylim=c(0,NA), cex.axis=0.75)

## Inverting predictors and predictands ("generative mode")

We can also infer `occupation` and `sex` given any value of `income`

In [ ]:
result <- infer(agent=opmall, predictand=c('capSurface', 'capColor'),
                predictor=list(class='e'))

print('e = edible')
print(round(result * 100, 2))

cat('\n\n') # some newlines


result <- infer(agent=opmall, predictand=c('capSurface', 'capColor'),
                predictor=list(class='p'))

print('p = poisonus')
print(round(result * 100, 2))

----

## The `rF()` function

This function generates **full-population** frequency distributions (even for subpopulations) that are probable according to the data.

For instance, let's see three samples of how the full-population frequency distribution for `sex` and `income` (jointly) could be:

In [ ]:
result <- rF(agent=opmall, n=3, predictand=c('capSurface', 'capColor'))

print(aperm(result) * 100) # permutation so that the samples are the last array dimension

These possible full-population frequency distributions can be used to assess how much the probabilities we find could change, if we collected a much, much larger amount of training data.

In the example below, we generate 1000 frequency distributions for `occupation` & `sex` given `income == >50K`, and then take the standard deviations of the samples as a rough measure of how much the probabilities we calculated a couple of cells above could change:

In [ ]:
freqsamples <- rF(n=1000, agent=opmall, predictand=c('capSurface', 'capColor'),
                  predictor=list(class='e'))

variability <- apply(freqsamples,
                     c('capSurface', 'capColor'), # which dimensions to apply
                     sd) # function to apply to those dimensions

print(round(variability * 100, 2)) # round to two decimals

----

## Calculating relevance of predictors

The agent can calculate the mutual information (measured in shannons) between any two sets of variates `A` and `B` of our choice, with the function `mutualinfo(probs, A, B)`. The joint probabilities for `(A,B)` must first be calculated with `infer()`.

For instance, what is the mutual information between `occupation` and `marital status`? The mutual information in this case can be anywhere between 0 Sh and 1 Sh 

(We print the result with 4 significant digits)

In [ ]:
probs <- infer(agent=opmall, predictand=c('class', 'capSurface', 'capColor'))

result <- mutualinfo(probs=probs, A='class', B=c('capSurface', 'capColor'))

print(paste0(signif(result, 4), ' Sh'))

Let’s consider a scenario where, in order to save resources, we can use *only one* variate in order to infer the income. Which of the other variates should we prefer?

We can calculate the mutual information between each of them and `income`:

In [ ]:
## list of all variates
variates <- names(dimnames(opmall$counts))

## list of all variates except 'income'
predictors <- variates[variates != 'class']

## prepare vector to contain the mutual information between each of these possible variates and 'income'
relevances <- numeric(length(predictors))
names(relevances) <- predictors

## calculate joint probability and then mutual information (shannons)
## of 'income' and another variate
for(var in predictors){
    probs <- infer(agent=opmall, predictand=c('class',var)) # calculation of joint probabilities
    relevances[var] <- mutualinfo(probs=probs, A='class', B=var) # calculation of mutual information
}

## output the mutual informations in decreasing order
print(round(sort(relevances, decreasing=TRUE), 4))

----

# Matrix




----

# Exercise

Now consider the scenario where we must *exclude one variate* from the eight predictors, or, equivalently, we can only use seven variates as predictors. Which variate should we exclude?

Prepare a script similar to the one above: it should calculate the mutual information between `income` and the other predictors but with one omitted, omitting each of the eight predictors in turn.

- Which single variate should not be omitted from the predictors? which single variate could be dropped?

- Do you obtain the same relevance ranking as in the “use-one-variate-only” scenario above?

**WARNING:** this computation could even take 10 min or more!

----


## Making decisions

The agent can make decisions with an arbitrary set of available choices and utilities. The function `decide()` is used for this. It takes as input the probabilities of the uncertain variate, which must first be calculated with `infer()`, and the utilities.

As an example, let's imagine that a bank has three loan options `loan_A`, `loan_B`, `loan_C` which yield different utilities depending on the person's income, which however is unknown (unrealistic, I know; but imagine it's the income in ten years).

Let's build the utility matrix first:

In [ ]:
## Warning: in R, matrices are filled column-wise: column 1 first, then column 2, then column 3, and so on
loanutilities <- matrix(c(-4,-1,0, 0,-1,-4), nrow=3, ncol=2, 
                  dimnames=list(loan=c('loan_A','loan_B','loan_C'), income=c('<=50K','>50K')))

print(loanutilities)

Now a `race='Black'`, `sex='Female'` person with `occupation='Sales'` asks for a loan. What should the bank decide?

We let the `opmall`-agent calculate the probabilities, and then make the optimal decision:

In [ ]:
incomeprobs <- infer(agent=opmall, predictand='income', predictor=list(race='Black', sex='Female', occupation='Sales'))

optimalchoice <- decide(probs=incomeprobs, utils=loanutilities)

print(incomeprobs)
cat('\n')
print(optimalchoice)              

Another customer comes in: a `race='White'`, `sex='Male'` person with `occupation='Exec-managerial'`. What's the optimal loan type for this customer?

In [ ]:
incomeprobs <- infer(agent=opmall, predictand='income', predictor=list(race='White', sex='Male', occupation='Exec-managerial'))

optimalchoice <- decide(probs=incomeprobs, utils=loanutilities)

print(incomeprobs)
cat('\n')
print(optimalchoice)

In [ ]:
testdata <- fread('test-income_data_example.csv', header=TRUE) # read test data
ntest <- nrow(testdata) # number of test data

testprobs <- numeric(ntest) # prepare vector of probabilities
testhits <- numeric(ntest) # prepare vector of hits

stopwatch <- Sys.time() # record time

for(i in 1:ntest){
    ## calculate probabilities given all variates except 'income'
    probs <- infer(agent=opmall, predictor=testdata[i, !'income'])

    ## store the probability for <=50K
    testprobs[i] <- probs['<=50K']

    ## decide on one value, check if decision == true_value, and store result
    chosenvalue <- decide(probs=probs)

    testhits[i] <- (chosenvalue == testdata[i, 'income'])
}

print(Sys.time() - stopwatch) # print computation time


## Histogram and average
thist(testprobs, n=seq(0,1,length.out=10), plot=TRUE,
      xlab='P(income = "<=50K")',
      ylab='frequency density in test set',
      main=paste0('accuracy: ', round(100*mean(testhits), 1), '%'))

In [ ]:
# navn <- colnames(data)
# navn <- navn[2:15]
# print(length(navn))

# i <- 1
# j <- 2

# result_list <- list()

# while (i != 13) {
#   while (j != 14) {

#     probs <- infer(agent = opmall, predictand = c("class", navn[i], navn[j]))
#     result <- mutualinfo(probs = probs, A = "class", B = c(navn[i], navn[j]))

#     result_list[[paste0(navn[i], " & ", navn[j])]] <- result
#     #print(paste0(signif(result, 4), ' Sh'))

#     j <- j + 1
#   }
#   j <- i + 2
#   i <- i + 1
# }
# sorted_result_list <- result_list[order(-unlist(result_list))]
# sorted_result_list
